In [1]:
#!/usr/bin/env python3

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
!pip install more_itertools

This file illustrates how you might experiment with the HMM interface.
You can paste these commands in at the Python prompt, or execute `test_ic.py` directly.
A notebook interface is nicer than the plain Python prompt, so we provide
a notebook version of this file as `test_ic.ipynb`, which you can open with
`jupyter` or with Visual Studio `code` (run it with the `nlp-class` kernel).

In [4]:
import logging, math, os
from pathlib import Path

In [5]:
import torch
from torch import tensor

In [6]:
from corpus import TaggedCorpus
from eval import model_cross_entropy, write_tagging
from hmm import HiddenMarkovModel
from crf_backprop import ConditionalRandomFieldBackprop

Set up logging.

In [7]:
log = logging.getLogger("test_ic")       # For usage, see findsim.py in earlier assignment.
logging.root.setLevel(level=logging.INFO)
logging.basicConfig(level=logging.INFO)  # could change INFO to DEBUG
# torch.autograd.set_detect_anomaly(True)    # uncomment to improve error messages from .backward(), but slows down

Switch working directory to the directory where the data live.  You may want to edit this line.

In [8]:
os.chdir("../data")

Get vocabulary and tagset from a supervised corpus.

In [9]:
icsup = TaggedCorpus(Path("icsup"), add_oov=False)
log.info(f"Ice cream vocabulary: {list(icsup.vocab)}")
log.info(f"Ice cream tagset: {list(icsup.tagset)}")

INFO:corpus:Read 40 tokens from icsup
INFO:corpus:Created 4 tag types
INFO:corpus:Created 5 word types
INFO:test_ic:Ice cream vocabulary: ['1', '2', '3', '_EOS_WORD_', '_BOS_WORD_']
INFO:test_ic:Ice cream tagset: ['C', 'H', '_EOS_TAG_', '_BOS_TAG_']


Two ways to look at the corpus ...

In [10]:
os.system("cat icsup")   # call the shell to look at the file directly

1/C 1/C 1/C 1/C 1/C 1/C 1/C 2/C 2/C 3/H
1/H 2/H 2/H 3/H 3/H 3/H 3/H 3/H 3/H 3/C
1/C 1/C 1/C 1/C 1/C 1/C 1/C 2/C 2/C 3/H
1/H 2/H 2/H 3/H 3/H 3/H 3/H 3/H 3/H 3/C


0

In [11]:
log.info(icsup)          # print the TaggedCorpus python object we constructed from it

INFO:test_ic:1/C 1/C 1/C 1/C 1/C 1/C 1/C 2/C 2/C 3/H
1/H 2/H 2/H 3/H 3/H 3/H 3/H 3/H 3/H 3/C
1/C 1/C 1/C 1/C 1/C 1/C 1/C 2/C 2/C 3/H
1/H 2/H 2/H 3/H 3/H 3/H 3/H 3/H 3/H 3/C


Make an HMM.

In [12]:
log.info("*** Hidden Markov Model (HMM) test\n")
hmm = HiddenMarkovModel(icsup.tagset, icsup.vocab)
# Change the transition/emission initial probabilities to match the ice cream spreadsheet,
# and test your implementation of the Viterbi algorithm.  Note that the spreadsheet 
# uses transposed versions of these matrices.
hmm.B = tensor([[0.7000, 0.2000, 0.1000],    # emission probabilities
                [0.1000, 0.2000, 0.7000],
                [0.0000, 0.0000, 0.0000],
                [0.0000, 0.0000, 0.0000]])
hmm.A = tensor([[0.8000, 0.1000, 0.1000, 0.0000],   # transition probabilities
                [0.1000, 0.8000, 0.1000, 0.0000],
                [0.0000, 0.0000, 0.0000, 0.0000],
                [0.5000, 0.5000, 0.0000, 0.0000]])
log.info("*** Current A, B matrices (using initalizations from the ice cream spreadsheet)")
hmm.printAB()

INFO:test_ic:*** Hidden Markov Model (HMM) test

INFO:test_ic:*** Current A, B matrices (using initalizations from the ice cream spreadsheet)


Transition matrix A:
	C	H	_EOS_TAG_	_BOS_TAG_
C	0.800	0.100	0.100	0.000
H	0.100	0.800	0.100	0.000
_EOS_TAG_	0.000	0.000	0.000	0.000
_BOS_TAG_	0.500	0.500	0.000	0.000

Emission matrix B:
	1	2	3
C	0.700	0.200	0.100
H	0.100	0.200	0.700
_EOS_TAG_	0.000	0.000	0.000
_BOS_TAG_	0.000	0.000	0.000




Try it out on the raw data from the spreadsheet, available in `icraw``.

In [13]:
log.info("*** Viterbi results on icraw with hard coded parameters")
icraw = TaggedCorpus(Path("icraw"), tagset=icsup.tagset, vocab=icsup.vocab)
write_tagging(hmm, icraw, Path("icraw_hmm.output"))  # calls hmm.viterbi_tagging on each sentence
os.system("cat icraw_hmm.output")   # print the file we just created, and remove it


INFO:test_ic:*** Viterbi results on icraw with hard coded parameters
100%|██████████| 1/1 [00:00<00:00,  8.21it/s]

2/H 3/H 3/H 2/H 3/H 2/H 3/H 2/H 2/H 3/H 1/H 3/H 3/H 1/C 1/C 1/C 2/C 1/C 1/C 1/C 3/C 1/C 2/C 1/C 1/C 1/C 2/H 3/H 3/H 2/H 3/H 2/H 2/H


0

Did the parameters that we guessed above get the "correct" answer, 
as revealed in `icdev`?

In [14]:
icdev = TaggedCorpus(Path("icdev"), tagset=icsup.tagset, vocab=icsup.vocab)
log.info(f"*** Compare to icdev corpus:\n{icdev}")
from eval import viterbi_error_rate
viterbi_error_rate(hmm, icdev, show_cross_entropy=False)

INFO:test_ic:*** Compare to icdev corpus:
2/H 3/H 3/H 2/H 3/H 2/H 3/H 2/H 2/H 3/H 1/C 3/C 3/C 1/C 1/C 1/C 2/C 1/C 1/C 1/C 3/C 1/C 2/C 1/C 1/C 1/C 2/H 3/H 3/H 2/H 3/H 2/H 2/H
100%|██████████| 1/1 [00:00<00:00, 11.34it/s]
INFO:eval:Tagging accuracy: all: 90.909%, seen: 90.909%, novel: nan%


0.09090909090909094

Now let's try your training code, running it on supervised data.
To test this, we'll restart from a random initialization.
(You could also try creating this new model with `unigram=true`, 
which will affect the rest of the notebook.)

In [15]:
hmm = HiddenMarkovModel(icsup.tagset, icsup.vocab)
log.info("*** A, B matrices as randomly initialized close to uniform")
hmm.printAB()

INFO:test_ic:*** A, B matrices as randomly initialized close to uniform


Transition matrix A:
	C	H	_EOS_TAG_	_BOS_TAG_
C	0.445	0.286	0.269	0.000
H	0.281	0.448	0.271	0.000
_EOS_TAG_	0.290	0.272	0.438	0.000
_BOS_TAG_	0.344	0.334	0.322	0.000

Emission matrix B:
	1	2	3
C	0.332	0.333	0.334
H	0.324	0.336	0.340
_EOS_TAG_	0.000	0.000	0.000
_BOS_TAG_	0.000	0.000	0.000




In [16]:
log.info("*** Supervised training on icsup")
cross_entropy_loss = lambda model: model_cross_entropy(model, icsup)
hmm.train(corpus=icsup, loss=cross_entropy_loss, tolerance=0.0001)
log.info("*** A, B matrices after training on icsup (should "
         "match initial params on spreadsheet [transposed])")
hmm.printAB()

INFO:test_ic:*** Supervised training on icsup
100%|██████████| 4/4 [00:00<00:00, 60.14it/s]
INFO:eval:Cross-entropy: 1.4117 nats (= perplexity 4.103)
100%|██████████| 4/4 [00:00<00:00, 34.63it/s]
INFO:hmm:Saved model to my_hmm.pkl
100%|██████████| 4/4 [00:00<00:00, 78.19it/s]
INFO:eval:Cross-entropy: 1.0584 nats (= perplexity 2.882)
100%|██████████| 4/4 [00:00<00:00, 50.17it/s]
INFO:hmm:Saved model to my_hmm.pkl
100%|██████████| 4/4 [00:00<00:00, 115.29it/s]
INFO:eval:Cross-entropy: 1.0584 nats (= perplexity 2.882)
INFO:hmm:Saved model to my_hmm.pkl
INFO:test_ic:*** A, B matrices after training on icsup (should match initial params on spreadsheet [transposed])


Transition matrix A:
	C	H	_EOS_TAG_	_BOS_TAG_
C	0.800	0.100	0.100	0.000
H	0.100	0.800	0.100	0.000
_EOS_TAG_	0.000	0.000	0.000	0.000
_BOS_TAG_	0.500	0.500	0.000	0.000

Emission matrix B:
	1	2	3
C	0.700	0.200	0.100
H	0.100	0.200	0.700
_EOS_TAG_	0.000	0.000	0.000
_BOS_TAG_	0.000	0.000	0.000




Now that we've reached the spreadsheet's starting guess, let's again tag
the spreadsheet "sentence" (that is, the sequence of ice creams) using the
Viterbi algorithm.

In [17]:
log.info("*** Viterbi results on icraw")
icraw = TaggedCorpus(Path("icraw"), tagset=icsup.tagset, vocab=icsup.vocab)
write_tagging(hmm, icraw, Path("icraw_hmm.output"))  # calls hmm.viterbi_tagging on each sentence
os.system("cat icraw_hmm.output")   # print the file we just created, and remove it

INFO:test_ic:*** Viterbi results on icraw
100%|██████████| 1/1 [00:00<00:00, 13.36it/s]

2/H 3/H 3/H 2/H 3/H 2/H 3/H 2/H 2/H 3/H 1/H 3/H 3/H 1/C 1/C 1/C 2/C 1/C 1/C 1/C 3/C 1/C 2/C 1/C 1/C 1/C 2/H 3/H 3/H 2/H 3/H 2/H 2/H


0

Next let's use the forward algorithm to see what the model thinks about 
the probability of the spreadsheet "sentence."

In [18]:
log.info("*** Forward algorithm on icraw (should approximately match iteration 0 "
             "on spreadsheet)")
for sentence in icraw:
    prob = math.exp(hmm.logprob(sentence, icraw))
    log.info(f"{prob} = p({sentence})")

INFO:test_ic:*** Forward algorithm on icraw (should approximately match iteration 0 on spreadsheet)
INFO:test_ic:9.127694257509654e-19 = p(2 3 3 2 3 2 3 2 2 3 1 3 3 1 1 1 2 1 1 1 3 1 2 1 1 1 2 3 3 2 3 2 2)


Finally, let's reestimate on the icraw data, as the spreadsheet does.
We'll evaluate as we go along on the *training* perplexity, and stop
when that has more or less converged.

In [19]:
log.info("*** Reestimating on icraw (perplexity should improve on every iteration)")
negative_log_likelihood = lambda model: model_cross_entropy(model, icraw)  # evaluate on icraw itself
hmm.train(corpus=icraw, loss=negative_log_likelihood, tolerance=0.0001)

INFO:test_ic:*** Reestimating on icraw (perplexity should improve on every iteration)
100%|██████████| 1/1 [00:00<00:00, 12.46it/s]
INFO:eval:Cross-entropy: 1.2217 nats (= perplexity 3.393)
100%|██████████| 1/1 [00:00<00:00,  6.39it/s]
INFO:hmm:Saved model to my_hmm.pkl
100%|██████████| 1/1 [00:00<00:00, 13.87it/s]
INFO:eval:Cross-entropy: 1.0807 nats (= perplexity 2.947)
100%|██████████| 1/1 [00:00<00:00,  5.40it/s]
INFO:hmm:Saved model to my_hmm.pkl
100%|██████████| 1/1 [00:00<00:00, 10.64it/s]
INFO:eval:Cross-entropy: 1.0576 nats (= perplexity 2.879)
100%|██████████| 1/1 [00:00<00:00,  4.32it/s]
INFO:hmm:Saved model to my_hmm.pkl
100%|██████████| 1/1 [00:00<00:00, 13.62it/s]
INFO:eval:Cross-entropy: 1.0486 nats (= perplexity 2.854)
100%|██████████| 1/1 [00:00<00:00,  7.40it/s]
INFO:hmm:Saved model to my_hmm.pkl
100%|██████████| 1/1 [00:00<00:00, 10.36it/s]
INFO:eval:Cross-entropy: 1.0438 nats (= perplexity 2.840)
100%|██████████| 1/1 [00:00<00:00,  5.92it/s]
INFO:hmm:Saved model to 

In [20]:
log.info("*** A, B matrices after reestimation on icraw"
         "should match final params on spreadsheet [transposed])")
hmm.printAB()

INFO:test_ic:*** A, B matrices after reestimation on icrawshould match final params on spreadsheet [transposed])


Transition matrix A:
	C	H	_EOS_TAG_	_BOS_TAG_
C	0.934	0.066	0.000	0.000
H	0.072	0.865	0.063	0.000
_EOS_TAG_	0.000	0.000	0.000	0.000
_BOS_TAG_	0.000	1.000	0.000	0.000

Emission matrix B:
	1	2	3
C	0.641	0.148	0.211
H	0.000	0.534	0.466
_EOS_TAG_	0.000	0.000	0.000
_BOS_TAG_	0.000	0.000	0.000




Now let's try out a randomly initialized CRF on the ice cream data. Notice how
the initialized A and B matrices now hold non-negative potentials,
rather than probabilities that sum to 1.

In [ ]:
log.info("*** Conditional Random Field (CRF) Backprop test\n")
crf = ConditionalRandomFieldBackprop(icsup.tagset, icsup.vocab)
log.info("*** Current A, B matrices (potentials from small random parameters)")
crf.printAB()

Now let's try your training code, running it on supervised data. To test this,
we'll restart from a random initialization. 

Note that the logger reports the CRF's *conditional* cross-entropy, 
log p(tags | words) / n.  This is much lower than the HMM's *joint* 
cross-entropy log p(tags, words) / n, but that doesn't mean the CRF
is worse at tagging.  The CRF is just predicting less information.

In [22]:
log.info("*** Supervised training on icsup")
cross_entropy_loss = lambda model: model_cross_entropy(model, icsup)
crf.train(corpus=icsup, loss=cross_entropy_loss, lr=0.1, tolerance=0.0001)
log.info("*** A, B matrices after training on icsup")
crf.printAB()

INFO:test_ic:*** Supervised training on icsup
100%|██████████| 4/4 [00:00<00:00, 235.43it/s]
INFO:eval:Cross-entropy: 0.6292 nats (= perplexity 1.876)
100%|██████████| 4/4 [00:00<00:00, 365.62it/s]
INFO:eval:Cross-entropy: 0.2034 nats (= perplexity 1.226)
100%|██████████| 4/4 [00:00<00:00, 173.86it/s]
INFO:eval:Cross-entropy: 0.2019 nats (= perplexity 1.224)
100%|██████████| 4/4 [00:00<00:00, 352.55it/s]
INFO:eval:Cross-entropy: 0.2016 nats (= perplexity 1.223)
100%|██████████| 4/4 [00:00<00:00, 376.65it/s]
INFO:eval:Cross-entropy: 0.2013 nats (= perplexity 1.223)
100%|██████████| 4/4 [00:00<00:00, 384.09it/s]
INFO:eval:Cross-entropy: 0.2022 nats (= perplexity 1.224)
INFO:hmm:Saved model to my_hmm.pkl
INFO:test_ic:*** A, B matrices after training on icsup


Transition matrix A:
	C	H	_EOS_TAG_	_BOS_TAG_
C	1.298	0.071	8.528	0.000
H	2.149	1.269	0.118	0.000
_EOS_TAG_	0.000	0.000	0.000	0.000
_BOS_TAG_	0.100	10.151	0.000	0.000

Emission matrix B:
	1	2	3
C	21.280	1.050	0.060
H	0.047	0.967	16.656
_EOS_TAG_	0.000	0.000	0.000
_BOS_TAG_	0.000	0.000	0.000




Let's again tag the spreadsheet "sentence" (that is, the sequence of ice creams) 
using the Viterbi algorithm (this may not match the HMM).

In [23]:
log.info("*** Viterbi results on icraw with trained parameters")
icraw = TaggedCorpus(Path("icraw"), tagset=icsup.tagset, vocab=icsup.vocab)
write_tagging(hmm, icraw, Path("icraw_crf.output"))  # calls hmm.viterbi_tagging on each sentence
os.system("cat icraw_crf.output")   # print the file we just created, and remove it

viterbi_error_rate(crfb, icdev, show_cross_entropy=False)

INFO:test_ic:*** Viterbi results on icraw with trained parameters
100%|██████████| 1/1 [00:00<00:00, 13.23it/s]


2/H 3/H 3/H 2/H 3/H 2/H 3/H 2/H 2/H 3/H 1/C 3/C 3/C 1/C 1/C 1/C 2/C 1/C 1/C 1/C 3/C 1/C 2/C 1/C 1/C 1/C 2/H 3/H 3/H 2/H 3/H 2/H 2/H


NameError: name 'crfb' is not defined

In [24]:
log.info("*** Viterbi results on icraw with trained parameters")
icraw = TaggedCorpus(Path("icraw"), tagset=icsup.tagset, vocab=icsup.vocab)
write_tagging(hmm, icraw, Path("icraw_crf.output"))  # calls hmm.viterbi_tagging on each sentence
os.system("cat icraw_crf.output")   # print the file we just created, and remove it

viterbi_error_rate(crf, icdev, show_cross_entropy=False)

INFO:test_ic:*** Viterbi results on icraw with trained parameters
100%|██████████| 1/1 [00:00<00:00,  5.27it/s]


2/H 3/H 3/H 2/H 3/H 2/H 3/H 2/H 2/H 3/H 1/C 3/C 3/C 1/C 1/C 1/C 2/C 1/C 1/C 1/C 3/C 1/C 2/C 1/C 1/C 1/C 2/H 3/H 3/H 2/H 3/H 2/H 2/H


100%|██████████| 1/1 [00:00<00:00,  8.25it/s]
INFO:eval:Tagging accuracy: all: 81.818%, seen: 81.818%, novel: nan%


0.18181818181818177

In [ ]:
import torch
from torch import Tensor, cuda, nn
from corpus import Sentence

class TestNonStationaryHMM(HiddenMarkovModel):
    """A simple non-stationary HMM that changes its parameters based on position."""
    
    def A_at(self, position: int, sentence: Sentence) -> Tensor:
        """Return different transition probabilities at different positions."""
        if position == 1:  # First real position after BOS
            # Higher probability of H→C at the start
            A = self.A.clone()
            valid_mask = torch.ones(self.k, dtype=torch.bool)
            valid_mask[self.bos_t] = False
            valid_mask[self.eos_t] = False
            A[valid_mask] *= 1.2  # Increase non-special transitions
            return A.softmax(dim=1)  # Renormalize
        return self.A

    def B_at(self, position: int, sentence: Sentence) -> Tensor:
        """Return different emission probabilities at different positions."""
        if position > 2:  # Later positions
            # Higher probability of emitting '3' later in sequence
            B = self.B.clone()
            B[:, 2] *= 1.2  # Increase probability of emitting '3'
            return B.softmax(dim=1)  # Renormalize
        return self.B

def test_nonstationary():
    """Test that non-stationary HMM behaves differently from regular HMM."""
    print("\n=== Testing Non-stationary HMM ===")
    
    # Create both types of HMMs with same initialization
    regular_hmm = HiddenMarkovModel(icsup.tagset, icsup.vocab)
    nonstat_hmm = TestNonStationaryHMM(icsup.tagset, icsup.vocab)
    
    # Set some specific initial parameters for clear testing
    regular_hmm.B = torch.tensor([[0.7, 0.2, 0.1],    # emission probabilities
                                 [0.1, 0.2, 0.7],
                                 [0.0, 0.0, 0.0],
                                 [0.0, 0.0, 0.0]])
    
    regular_hmm.A = torch.tensor([[0.8, 0.1, 0.1, 0.0],   # transition probabilities
                                 [0.1, 0.8, 0.1, 0.0],
                                 [0.0, 0.0, 0.0, 0.0],
                                 [0.5, 0.5, 0.0, 0.0]])
    
    # Copy parameters to non-stationary HMM
    nonstat_hmm.A = regular_hmm.A.clone()
    nonstat_hmm.B = regular_hmm.B.clone()
    
    # Get test sentence from corpus
    test_sent = next(iter(icdev))
    
    # Get probabilities for same sequence from both models
    reg_prob = regular_hmm.logprob(test_sent, icdev)
    nonstat_prob = nonstat_hmm.logprob(test_sent, icdev)
    
    # Print results
    print(f"\nTest sentence: {test_sent}")
    print(f"Regular HMM log probability: {reg_prob:.4f}")
    print(f"Non-stationary HMM log probability: {nonstat_prob:.4f}")
    
    # Compare taggings
    reg_tagging = regular_hmm.viterbi_tagging(test_sent, icdev)
    nonstat_tagging = nonstat_hmm.viterbi_tagging(test_sent, icdev)
    
    print(f"\nRegular HMM tagging: {reg_tagging}")
    print(f"Non-stationary HMM tagging: {nonstat_tagging}")
    
    # Verify different probabilities
    assert reg_prob != nonstat_prob, "Non-stationary HMM gave same probability as regular HMM"
    print("\nTest passed: Non-stationary HMM produced different probabilities as expected")


# Add this to your existing test code
if __name__ == "__main__":
    # ... your existing test code ...
    
    # Test non-stationary modifications
    test_nonstationary()